In [ ]:
#-------------------------- Standard Imports --------------------------#
%reload_ext autoreload
%autoreload 2
import kdephys as kde
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import acr

# ---------------------------- EXTRAS --------------------------------#
from kdephys.plot.main import _title, bp_plot
import kdephys.utils.spectral as sp
from acr.utils import *
bands = sp.bands

import pingouin as pg
from scipy import stats
from scipy.stats import shapiro

from acr.plots import pub, lrg
plt.style.use('fast')
lrg()
plt.rcParams['xtick.bottom'] = False

import matplotlib as mpl
import brokenaxes as bx

#--------------------------------- Import Publication Functions ---------------------------------#
pub_utils = acr.utils.import_publication_functions('/home/kdriessen/gh_master/PUBLICATION__ACR/pub_utils.py', 'pub_utils')
data_agg = acr.utils.import_publication_functions('/home/kdriessen/gh_master/PUBLICATION__ACR/data_agg.py', 'data_agg')

In [ ]:
from scipy.stats import sem
def mean_and_sem(data):
    return np.mean(data), sem(data)

In [ ]:
molecular_path = '/Volumes/opto_loc/Data/molecular_results.xlsx'
mdf = pd.read_excel(molecular_path)

In [ ]:
master_fig_size = (3, 4)

# pGluR1

In [ ]:
som_pglu = mdf.query("type == 'som'").query("measure == 'pGluR1_GAPDH'")
som_trt = np.array(som_pglu.query("hemi == 'T'")['val'])
som_con = np.array(som_pglu.query("hemi == 'C'")['val'])
acr_pglu = mdf.query("type == 'acr'").query("measure == 'pGluR1_GAPDH'")
acr_trt = np.array(acr_pglu.query("hemi == 'T'")['val'])
acr_con = np.array(acr_pglu.query("hemi == 'C'")['val'])
control_pglu = mdf.query("type == 'control'").query("measure == 'pGluR1_GAPDH'")
control_trt = np.array(control_pglu.query("hemi == 'T'")['val'])
control_con = np.array(control_pglu.query("hemi == 'C'")['val'])

In [ ]:
# Normality Check
s, p = shapiro((som_con-som_trt))
print(f'shapiro p-val, som mice: {p}')
s, p = shapiro((acr_con-acr_trt))
print(f'shapiro p-val, acr mice: {p}')
s, p = shapiro((control_con-control_trt))
print(f'shapiro p-val, control mice: {p}')

In [ ]:
stats = pg.ttest(som_con, som_trt, paired=True)
hg = pg.compute_effsize(som_con, som_trt, paired=True, eftype='hedges')
print(f'hedges g: {hg}')

# ==== Write Stats Results ====
stats_name = 'SOM_pglur1'
acr.stats.write_stats_result(stats_name, 'paired_ttest', stats['T'][0], stats['p-val'][0], 'g', hg)

stats

In [ ]:
stats = pg.ttest(acr_con, acr_trt, paired=True)
hg = pg.compute_effsize(acr_con, acr_trt, paired=True, eftype='hedges')
print(f'hedges g: {hg}')

# ==== Write Stats Results ====
stats_name = 'ACR_pglur1'
acr.stats.write_stats_result(stats_name, 'paired_ttest', stats['T'][0], stats['p-val'][0], 'g', hg)

stats

In [ ]:
stats = pg.wilcoxon(control_con, control_trt)
hg = pg.compute_effsize(control_con, control_trt, paired=True, eftype='hedges')
print(f'hedges g: {hg}')
r = acr.stats.calculate_wilx_r(stats['W-val'][0], len(control_con))

# ==== Write Stats Results ====
stats_name = 'CONTROL_pglur1'
acr.stats.write_stats_result(stats_name, 'wilcoxon', stats['W-val'][0], stats['p-val'][0], 'r, RBC', [r, stats['RBC'][0]])

stats

In [ ]:
# write source data
trt_df = pd.DataFrame({'type': ['som', 'acr', 'control'], 'hemi': ['off-induction', 'off-induction', 'off-induction'], 'values': [som_trt, acr_trt, control_trt]})
con_df = pd.DataFrame({'type': ['som', 'acr', 'control'], 'hemi': ['contralateral', 'contralateral', 'contralateral'], 'values': [som_con, acr_con, control_con]})
pglur1_df = pd.concat([trt_df, con_df])
pub_utils.write_source_data(pglur1_df, 'pglur1_source_data')

In [ ]:
# Normalize by contra
som_trt = som_trt / som_con
som_con = np.ones(len(som_con))

acr_trt = acr_trt / acr_con
acr_con = np.ones(len(acr_con))

control_trt = control_trt / control_con
control_con = np.ones(len(control_con))

In [ ]:
som_con_mean, som_con_sem = mean_and_sem(som_con)
som_trt_mean, som_trt_sem = mean_and_sem(som_trt)

acr_con_mean, acr_con_sem = mean_and_sem(acr_con)
acr_trt_mean, acr_trt_sem = mean_and_sem(acr_trt)

control_con_mean, control_con_sem = mean_and_sem(control_con)
control_trt_mean, control_trt_sem = mean_and_sem(control_trt)

In [ ]:
f, ax = plt.subplots(figsize=master_fig_size)
x_pos = [1, 1.5]

for i in range(8):
    ax.plot(x_pos, [som_con[i], som_trt[i]],
             color=SOM_BLUE, alpha=0.8, linewidth=4)


ax.bar(x_pos, [som_con_mean, som_trt_mean], color=SOM_BLUE, alpha=0.4, width=0.2, yerr=[som_con_sem, som_trt_sem], error_kw=dict(ecolor='k', lw=4, capsize=0, capthick=0))

ax.set_ylim(0.4, 1.02)
ax.set_yticks([0.4, 0.7, 1])

ax.set_xlim(0.75, 1.75)
plt.tight_layout(pad=0.1)
plt.savefig(f'{PAPER_FIGURE_ROOT}/molecular/som-pglu.png', bbox_inches='tight', transparent=True, dpi=600)

In [ ]:
f, ax = plt.subplots(figsize=master_fig_size)
x_pos = [1, 1.5]

for i in range(8):
    ax.plot(x_pos, [acr_con[i], acr_trt[i]],
             color=ACR_BLUE, alpha=1, linewidth=4)


ax.bar(x_pos, [acr_con_mean, acr_trt_mean], color=ACR_BLUE, alpha=0.5, width=0.2, yerr=[acr_con_sem, acr_trt_sem], error_kw=dict(ecolor='k', lw=4, capsize=0, capthick=0))

ax.set_ylim(0.4, 1.02)
ax.set_yticks([0.4, 0.7, 1])

ax.set_xlim(0.75, 1.75)
plt.tight_layout(pad=0.1)
plt.savefig(f'{PAPER_FIGURE_ROOT}/molecular/acr-pglu.png', bbox_inches='tight', transparent=True, dpi=600)

In [ ]:
f, ax = plt.subplots(figsize=master_fig_size)
x_pos = [1, 1.5]

for i in range(8):
    ax.plot(x_pos, [control_con[i], control_trt[i]],
             color=NNXR_GRAY, alpha=1, linewidth=4)


ax.bar(x_pos, [control_con_mean, control_trt_mean], color=NNXR_GRAY, alpha=0.4, width=0.2, yerr=[control_con_sem, control_trt_sem], error_kw=dict(ecolor='k', lw=4, capsize=0, capthick=0))

ax.set_yticks([0.4, 1, 1.5])
ax.set_ylim(0.4, 1.55)

ax.set_xlim(0.75, 1.75)
plt.tight_layout(pad=0.1)
plt.savefig(f'{PAPER_FIGURE_ROOT}/molecular/control-pglu.png', bbox_inches='tight', transparent=True, dpi=600)

# GluR1 ONLY

In [ ]:
som_pglu = mdf.query("type == 'som'").query("measure == 'GluR1_GAPDH'")
som_trt = np.array(som_pglu.query("hemi == 'T'")['val'])
som_con = np.array(som_pglu.query("hemi == 'C'")['val'])
acr_pglu = mdf.query("type == 'acr'").query("measure == 'GluR1_GAPDH'")
acr_trt = np.array(acr_pglu.query("hemi == 'T'")['val'])
acr_con = np.array(acr_pglu.query("hemi == 'C'")['val'])
control_pglu = mdf.query("type == 'control'").query("measure == 'GluR1_GAPDH'")
control_trt = np.array(control_pglu.query("hemi == 'T'")['val'])
control_con = np.array(control_pglu.query("hemi == 'C'")['val'])

In [ ]:
# Normality Check
s, p = shapiro((som_con-som_trt))
print(f'shapiro p-val, som mice: {p}')
s, p = shapiro((acr_con-acr_trt))
print(f'shapiro p-val, acr mice: {p}')
s, p = shapiro((control_con-control_trt))
print(f'shapiro p-val, control mice: {p}')

In [ ]:
stats = pg.ttest(som_con, som_trt, paired=True)
hg = pg.compute_effsize(som_con, som_trt, paired=True, eftype='hedges')
print(f'hedges g: {hg}')

# ==== Write Stats Results ====
stats_name = 'SOM_Glur1'
acr.stats.write_stats_result(stats_name, 'paired_ttest', stats['T'][0], stats['p-val'][0], 'g', hg)

stats

In [ ]:
stats = pg.ttest(acr_con, acr_trt, paired=True)
hg = pg.compute_effsize(acr_con, acr_trt, paired=True, eftype='hedges')
print(f'hedges g: {hg}')

# ==== Write Stats Results ====
stats_name = 'ACR_Glur1'
acr.stats.write_stats_result(stats_name, 'paired_ttest', stats['T'][0], stats['p-val'][0], 'g', hg)

stats

In [ ]:
stats = pg.ttest(control_con, control_trt, paired=True)
hg = pg.compute_effsize(control_con, control_trt, paired=True, eftype='hedges')
print(f'hedges g: {hg}')

# ==== Write Stats Results ====
stats_name = 'CONTROL_Glur1'
acr.stats.write_stats_result(stats_name, 'paired_ttest', stats['T'][0], stats['p-val'][0], 'g', hg)

stats

In [ ]:
# write source data
trt_df = pd.DataFrame({'type': ['som', 'acr', 'control'], 'hemi': ['off-induction', 'off-induction', 'off-induction'], 'values': [som_trt, acr_trt, control_trt]})
con_df = pd.DataFrame({'type': ['som', 'acr', 'control'], 'hemi': ['contralateral', 'contralateral', 'contralateral'], 'values': [som_con, acr_con, control_con]})
glur1_df = pd.concat([trt_df, con_df])
pub_utils.write_source_data(glur1_df, 'glur1_source_data')

In [ ]:
som_trt = som_trt / som_con
som_con = np.ones(len(som_con))
acr_trt = acr_trt / acr_con
acr_con = np.ones(len(acr_con))
control_trt = control_trt / control_con
control_con = np.ones(len(control_con))

In [ ]:
som_con_mean, som_con_sem = mean_and_sem(som_con)
som_trt_mean, som_trt_sem = mean_and_sem(som_trt)

acr_con_mean, acr_con_sem = mean_and_sem(acr_con)
acr_trt_mean, acr_trt_sem = mean_and_sem(acr_trt)

control_con_mean, control_con_sem = mean_and_sem(control_con)
control_trt_mean, control_trt_sem = mean_and_sem(control_trt)

In [ ]:
f, ax = plt.subplots(figsize=master_fig_size)
x_pos = [1, 1.5]

for i in range(8):
    ax.plot(x_pos, [som_con[i], som_trt[i]],
             color=SOM_BLUE, alpha=0.8, linewidth=4)


ax.bar(x_pos, [som_con_mean, som_trt_mean], color=SOM_BLUE, alpha=0.4, width=0.2, yerr=[som_con_sem, som_trt_sem], error_kw=dict(ecolor='k', lw=4, capsize=0, capthick=0))

ax.set_ylim(0.4, 1.02)
ax.set_yticks([0.4, 0.7, 1])

ax.set_xlim(0.75, 1.75)
plt.tight_layout(pad=0.1)
plt.savefig(f'{PAPER_FIGURE_ROOT}/molecular/som-glu.png', bbox_inches='tight', transparent=True, dpi=600)

In [ ]:
f, ax = plt.subplots(figsize=master_fig_size)
x_pos = [1, 1.5]

for i in range(8):
    ax.plot(x_pos, [acr_con[i], acr_trt[i]],
             color=ACR_BLUE, alpha=1, linewidth=4)


ax.bar(x_pos, [acr_con_mean, acr_trt_mean], color=ACR_BLUE, alpha=0.5, width=0.2, yerr=[acr_con_sem, acr_trt_sem], error_kw=dict(ecolor='k', lw=4, capsize=0, capthick=0))

ax.set_ylim(0.4, 1.02)
ax.set_yticks([0.4, 0.7, 1])

ax.set_xlim(0.75, 1.75)
plt.tight_layout(pad=0.1)
plt.savefig(f'{PAPER_FIGURE_ROOT}/molecular/acr-glu.png', bbox_inches='tight', transparent=True, dpi=600)

In [ ]:
f, ax = plt.subplots(figsize=master_fig_size)
x_pos = [1, 1.5]

for i in range(8):
    ax.plot(x_pos, [control_con[i], control_trt[i]],
             color=NNXR_GRAY, alpha=1, linewidth=4)


ax.bar(x_pos, [control_con_mean, control_trt_mean], color=NNXR_GRAY, alpha=0.4, width=0.2, yerr=[control_con_sem, control_trt_sem], error_kw=dict(ecolor='k', lw=4, capsize=0, capthick=0))

ax.set_ylim(0.4, 1.12)
ax.set_yticks([0.4, 0.7, 1.0])

ax.set_xlim(0.75, 1.75)
plt.tight_layout(pad=0.1)
plt.savefig(f'{PAPER_FIGURE_ROOT}/molecular/control-glu.png', bbox_inches='tight', transparent=True, dpi=600)

# RATIO

In [ ]:
som_pglu = mdf.query("type == 'som'").query("measure == 'pGluR1_GluR1'")
som_trt = np.array(som_pglu.query("hemi == 'T'")['val'])
som_con = np.array(som_pglu.query("hemi == 'C'")['val'])
acr_pglu = mdf.query("type == 'acr'").query("measure == 'pGluR1_GluR1'")
acr_trt = np.array(acr_pglu.query("hemi == 'T'")['val'])
acr_con = np.array(acr_pglu.query("hemi == 'C'")['val'])
control_pglu = mdf.query("type == 'control'").query("measure == 'pGluR1_GluR1'")
control_trt = np.array(control_pglu.query("hemi == 'T'")['val'])
control_con = np.array(control_pglu.query("hemi == 'C'")['val'])

In [ ]:
# Normality Check
s, p = shapiro((som_con-som_trt))
print(f'shapiro p-val, som mice: {p}')
s, p = shapiro((acr_con-acr_trt))
print(f'shapiro p-val, acr mice: {p}')
s, p = shapiro((control_con-control_trt))
print(f'shapiro p-val, control mice: {p}')

In [ ]:
stats = pg.ttest(som_con, som_trt, paired=True)
hg = pg.compute_effsize(som_con, som_trt, paired=True, eftype='hedges')
print(f'hedges g: {hg}')

# ==== Write Stats Results ====
stats_name = 'SOM_ratio'
acr.stats.write_stats_result(stats_name, 'paired_ttest', stats['T'][0], stats['p-val'][0], 'g', hg)

stats

In [ ]:
stats = pg.ttest(acr_con, acr_trt, paired=True)
hg = pg.compute_effsize(acr_con, acr_trt, paired=True, eftype='hedges')
print(f'hedges g: {hg}')

# ==== Write Stats Results ====
stats_name = 'ACR_ratio'
acr.stats.write_stats_result(stats_name, 'paired_ttest', stats['T'][0], stats['p-val'][0], 'g', hg)

stats

In [ ]:
stats = pg.ttest(control_con, control_trt, paired=True)
hg = pg.compute_effsize(control_con, control_trt, paired=True, eftype='hedges')
print(f'hedges g: {hg}')

# ==== Write Stats Results ====
stats_name = 'CONTROL_ratio'
acr.stats.write_stats_result(stats_name, 'paired_ttest', stats['T'][0], stats['p-val'][0], 'g', hg)

stats

In [ ]:
# write source data
trt_df = pd.DataFrame({'type': ['som', 'acr', 'control'], 'hemi': ['off-induction', 'off-induction', 'off-induction'], 'values': [som_trt, acr_trt, control_trt]})
con_df = pd.DataFrame({'type': ['som', 'acr', 'control'], 'hemi': ['contralateral', 'contralateral', 'contralateral'], 'values': [som_con, acr_con, control_con]})
ratio_df = pd.concat([trt_df, con_df])
pub_utils.write_source_data(ratio_df, 'ratio_source_data')

In [ ]:
som_trt = som_trt / som_con
som_con = np.ones(len(som_con))
acr_trt = acr_trt / acr_con
acr_con = np.ones(len(acr_con))
control_trt = control_trt / control_con
control_con = np.ones(len(control_con))

In [ ]:
som_con_mean, som_con_sem = mean_and_sem(som_con)
som_trt_mean, som_trt_sem = mean_and_sem(som_trt)

acr_con_mean, acr_con_sem = mean_and_sem(acr_con)
acr_trt_mean, acr_trt_sem = mean_and_sem(acr_trt)

control_con_mean, control_con_sem = mean_and_sem(control_con)
control_trt_mean, control_trt_sem = mean_and_sem(control_trt)

In [ ]:
f, ax = plt.subplots(figsize=master_fig_size)
x_pos = [1, 1.5]

for i in range(8):
    ax.plot(x_pos, [som_con[i], som_trt[i]],
             color=SOM_BLUE, alpha=0.8, linewidth=4)


ax.bar(x_pos, [som_con_mean, som_trt_mean], color=SOM_BLUE, alpha=0.4, width=0.2, yerr=[som_con_sem, som_trt_sem], error_kw=dict(ecolor='k', lw=4, capsize=0, capthick=0))

ax.set_ylim(0.6, 1.2)
ax.set_yticks([0.6, 1, 1.2])

ax.set_xlim(0.75, 1.75)
plt.tight_layout(pad=0.1)
plt.savefig(f'{PAPER_FIGURE_ROOT}/molecular/som-ratio.png', bbox_inches='tight', transparent=True, dpi=600)

In [ ]:
f, ax = plt.subplots(figsize=master_fig_size)
x_pos = [1, 1.5]

for i in range(8):
    ax.plot(x_pos, [acr_con[i], acr_trt[i]],
             color=ACR_BLUE, alpha=1, linewidth=4)


ax.bar(x_pos, [acr_con_mean, acr_trt_mean], color=ACR_BLUE, alpha=0.5, width=0.2, yerr=[acr_con_sem, acr_trt_sem], error_kw=dict(ecolor='k', lw=4, capsize=0, capthick=0))

ax.set_ylim(0.6, 1.4)
ax.set_yticks([0.6, 1, 1.4])

ax.set_xlim(0.75, 1.75)
plt.tight_layout(pad=0.1)
plt.savefig(f'{PAPER_FIGURE_ROOT}/molecular/acr-ratio.png', bbox_inches='tight', transparent=True, dpi=600)

In [ ]:
f, ax = plt.subplots(figsize=master_fig_size)
x_pos = [1, 1.5]

for i in range(8):
    ax.plot(x_pos, [control_con[i], control_trt[i]],
             color=NNXR_GRAY, alpha=1, linewidth=4)


ax.bar(x_pos, [control_con_mean, control_trt_mean], color=NNXR_GRAY, alpha=0.4, width=0.2, yerr=[control_con_sem, control_trt_sem], error_kw=dict(ecolor='k', lw=4, capsize=0, capthick=0))

ax.set_ylim(0.6, 1.5)
ax.set_yticks([0.6, 1, 1.5])

ax.set_xlim(0.75, 1.75)
plt.tight_layout(pad=0.1)
plt.savefig(f'{PAPER_FIGURE_ROOT}/molecular/control-ratio.png', bbox_inches='tight', transparent=True, dpi=600)